## TODOs
* in compose_tfms2 return a custom type
* this custom type should handle ToTensor

In [ ]:
from fastai2.basics import *
from fastai2.text.all import *
from pigboat.text.all import *

In [ ]:
from snorkel.labeling import model

In [ ]:
source = untar_data(URLs.IMDB_SAMPLE)

In [ ]:
df = pd.read_csv(source/'texts.csv')

In [ ]:
splits = ColSplitter('is_valid')(df)

In [ ]:
POS,NEG = 'positive','negative'
vocab = ABSTAIN,POS,NEG

In [ ]:
POS_WORDS  = ['enjoy', 'liked', 'love', 'masterpiece', 'enjoyed', 'loved', 'nice', 'good acting']
NEG_WORDS = ['hate', 'poor', 'bad', 'hated', 'dislike', 'disliked', 'suck', 'lame', 'worst']

In [ ]:
lblr = []
lblr += [WordLabeller(w, POS) for w in POS_WORDS]
lblr += [WordLabeller(w, NEG) for w in NEG_WORDS]

In [ ]:
class Numericalize2(Numericalize):
    def setups(self, items):
        set_trace()
        super().setups(items)

In [ ]:
dset_items = Datasets(df, [[ColReader('text'), Tokenizer.from_df('text'), Numericalize2()],
                           [ColReader('label'), Categorize]], splits=splits)

> <ipython-input-17-396fa7a48168>(4)setups()
-> super().setups(items)
(Pdb) items.text
*** AttributeError: text
(Pdb) items[0].text
*** AttributeError: 'Tokens' object has no attribute 'text'
(Pdb) len(items)
800
(Pdb) items
TfmdLists:         label  is_valid  \
0    negative     False   
1    positive     False   
2    negative     False   
3    positive     False   
4    negative     False   
..        ...       ...   
795  negative     False   
796  negative     False   
797  positive     False   
798  negative     False   
799  negative     False   

                                                                                                                                                                                                                                                                                                                                                                                                                                                       

(Pdb) type(items)
<class 'fastai2.data.core.TfmdLists'>
(Pdb) len(items)
800
(Pdb) items[0].text
*** AttributeError: 'Tokens' object has no attribute 'text'
(Pdb) items[0]
(#108) ['xxbos','xxmaj','un','-','bleeping','-','believable','!','xxmaj','meg'...]
(Pdb) exit


BdbQuit: 

In [ ]:
pipe = Pipeline2([ColReader('text'), Tokenizer.from_df('text'), Numericalize2()], lblr)
pipe.order = 0
pipe.name = 'Pipeline2'

In [ ]:
TfmdLists.setup??

In [ ]:
get_x = [pipe, Categorize2(vocab=vocab)]
# get_y = [ColReader('label'), Categorize(vocab=vocab)]
dset_train = Datasets(df, [get_x], splits=splits).train
# dset_valid = Datasets(df, [get_x, get_y], splits=splits, do_setup=False).valid

> <ipython-input-17-396fa7a48168>(4)setups()
-> super().setups(items)
(Pdb) items[0]
label                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         negative
is_valid                                                                                                                                                                                                                                                                                                    

BdbQuit: 

In [ ]:
%debug

> /home/lgvaz/anaconda3/envs/dl/lib/python3.7/collections/__init__.py(655)update()
    653                     super(Counter, self).update(iterable) # fast path when counter is empty
    654             else:
--> 655                 _count_elements(self, iterable)
    656         if kwds:
    657             self.update(kwds)

ipdb> u
> /home/lgvaz/anaconda3/envs/dl/lib/python3.7/collections/__init__.py(568)__init__()
    566             raise TypeError('expected at most 1 arguments, got %d' % len(args))
    567         super(Counter, self).__init__()
--> 568         self.update(*args, **kwds)
    569 
    570     def __missing__(self, key):

ipdb> u
> /home/lgvaz/git/fastai2/fastai2/text/data.py(38)setups()
     36         if dsets is None: return
     37         if self.vocab is None:
---> 38             count = dsets.counter if hasattr(dsets, 'counter') else Counter(p for o in dsets for p in o)
     39             self.vocab = make_vocab(count, min_freq=self.min_freq, max_vocab=self

In [ ]:
dset_train[0]

In [ ]:
dls = DataLoaders.from_dsets(dset_train, dset_valid, shuffle=False, device=default_device(), drop_last=False)

In [ ]:
m = MajorityLabelVoter()

In [ ]:
def acc(inp,targ): return (inp==targ).float().mean()

In [ ]:
learn = Learner(dls, m, loss_func=lambda *_: tensor(0.), metrics=acc)

In [ ]:
preds,_ = learn.get_preds(dl=dls.valid)

In [ ]:
learn.validate()

(#2) [0.0,0.46000000834465027]

In [ ]:
(preds[idxs]==lbls_true[idxs]).float().mean()

tensor(0.7080)

In [ ]:
# How to apply in parellel
task_lbls = torch.stack([o.data for o in tls])

In [ ]:
# lbls[lbls==vocab.o2i[ABSTAIN]] = -1
# Only works because ABSTAIN is already 0
task_lbls -= 1

In [ ]:
task_lbls_arr = np.array(task_lbls)

In [ ]:
# lbl_m = LabelModel(2, verbose=True)
lbl_m = MajorityLabelVoter()
# lbl_m.fit(task_lbls_arr, n_epochs=500, log_freq=100)

In [ ]:
lbl_m.score(task_lbls_arr, lbls_true.numpy()-1)

{'accuracy': 0.7165775401069518}

In [ ]:
dls = tls.dataloaders(shuffe=False)

In [ ]:
b = dls.one_batch()

In [ ]:
len(b)

3

In [ ]:
ToTensor()(items[0])

TensorCategory(0)